In [1]:
import numpy as np
import yfinance as yf
import pandas as pd
import requests
import re
import random
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from requests.exceptions import ConnectionError, Timeout, RequestException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import datetime
import numpy as np
from extract_options_chain import ExtractOptionsChain
from extract_options_data import ExtractOptionsData
from estimate_volatility import EstimateVolatility
from extract_rf import extract_risk_free_rate
import math
import pandas as pd
from greeks import Extract_Greeks
from options_strategies import Strategies
import pickle

In [ ]:
url = "https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option('detach', True)
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.TAG_NAME, "body"))
                )
json_text = driver.find_element(By.TAG_NAME, "body").text
data = json.loads(json_text)
print("✅ Data fetched successfully!")
driver.quit()

In [ ]:
records = data.get("records", {}).get("data", [])
calls = [d["CE"] for d in records if "CE" in d]
puts = [d["PE"] for d in records if "PE" in d]

In [ ]:
call_data = [
            (
                d.get('underlying', 'N/A'),  # Symbol
                d.get('strikePrice', 0),  # Strike Price
                pd.to_datetime(d.get('expiryDate', '1970-01-01'), errors='coerce').date(),  # Expiry Date
                d.get('bidQty', 0),  # Bid Quantity
                d.get('bidprice', 0),  # Bid Price
                d.get('askQty', 0),  # Ask Quantity
                d.get('askPrice', 0),  # Ask Price
                d.get('underlyingValue', 0),  # underlying's current price
            )
            for d in calls
        ]
call_df = pd.DataFrame(call_data, columns=[
            'symbol', 'strikePrice', 'expiryDate', 'bid_qty', 'bid_price',
            'ask_qty', 'ask_price', 'ltp'
        ])
call_df['S-K'] = np.abs(call_df['ltp'] - call_df['strikePrice'])
current_date = np.datetime64(datetime.date.today(), 'D')
expiry_dates_np = call_df['expiryDate'].values.astype('datetime64[D]')
call_df['calendar_days_to_maturity'] = np.busday_count(current_date, expiry_dates_np, weekmask='1111111')
call_df = call_df[(call_df['calendar_days_to_maturity'] >= 30) & (call_df['calendar_days_to_maturity'] <= 60)]
call_df['mkt_price'] = (call_df['bid_price'] + call_df['ask_price'])/2
call_df = call_df.sort_values(by='S-K')

In [ ]:
put_data = [
            (
                d.get('underlying', 'N/A'),  # Symbol
                d.get('strikePrice', 0),  # Strike Price
                pd.to_datetime(d.get('expiryDate', '1970-01-01'), errors='coerce').date(),  # Expiry Date
                d.get('bidQty', 0),  # Bid Quantity
                d.get('bidprice', 0),  # Bid Price
                d.get('askQty', 0),  # Ask Quantity
                d.get('askPrice', 0),  # Ask Price
                d.get('underlyingValue', 0),  # Possibly the underlying's current price, not the option's LTP
            )
            for d in puts
        ]

# Convert the list of tuples into a DataFrame
put_df = pd.DataFrame(put_data, columns=[
    'symbol', 'strikePrice', 'expiryDate', 'bid_qty', 'bid_price',
    'ask_qty', 'ask_price', 'ltp'
])

# Create a column 'K-S' as the absolute difference between 'strike' and 'ltp'
put_df['K-S'] = np.abs(put_df['strikePrice'] - put_df['ltp'])

# Get the current date and convert it to numpy.datetime64 with daily resolution
current_date = np.datetime64(datetime.date.today(), 'D')
# Convert the 'expiry_date' column to a numpy array of datetime64[D]
expiry_dates_np = put_df['expiryDate'].values.astype('datetime64[D]')
# Compute the number of business days (with all days considered as business days using weekmask='1111111')
put_df['calendar_days_to_maturity'] = np.busday_count(current_date, expiry_dates_np, weekmask='1111111')
put_df = put_df[(put_df['calendar_days_to_maturity'] >= 30) & (put_df['calendar_days_to_maturity'] <= 60)]
put_df['mkt_price'] = (put_df['bid_price'] + put_df['ask_price']) / 2
put_df = put_df.sort_values(by='K-S')

In [ ]:
call_df.to_csv('./calls.csv')
put_df.to_csv('./puts.csv')

In [ ]:
put_df

In [ ]:
pd.set_option('display.max_rows', 100)
from yahooquery import Ticker
ticker = Ticker('^NSEI', asynchronous=True, retry=20, status_forcelist=[404, 429, 500, 502, 503, 504])
daily_data = ticker.history(period='5y', interval='1d', adj_ohlc=True)
daily_data.index = daily_data.index.droplevel('symbol')
daily_data.to_csv('./daily_data.csv')

In [ ]:
high_frequency_data = ticker.history(period='30d', interval='15m', adj_ohlc=True)
high_frequency_data.index = high_frequency_data.index.droplevel('symbol')
high_frequency_data.to_csv('./high_frequency_data.csv')

In [ ]:
url = "https://www.fbil.org.in/#/home"
data = []
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option('detach', True)
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
time.sleep(3)
WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.LINK_TEXT, "MONEY MARKET/INTEREST RATES"))
            ).click()
WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.LINK_TEXT, "Term MIBOR"))
            ).click()
table = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "termMibor"))
            )
print("✅ Data fetched successfully!")
rows = table.find_elements(By.TAG_NAME, "tr")
tenor_mapping = {
                "14 DAYS": 14,
                "1 MONTH": 30,
                "3 MONTHS": 90
            }
for row in rows[1:]:
    cols = row.find_elements(By.TAG_NAME, "td")
    if len(cols) < 4:
        continue  # Skip rows that don't have enough columns

    date = cols[0].text.strip()
    tenor_text = cols[1].text.strip()
    rate = cols[3].text.strip()

    # Convert tenor to numeric days if available in our mapping
    if tenor_text in tenor_mapping:
        data.append([date, tenor_mapping[tenor_text], float(rate)])

# Convert the extracted data to a DataFrame
df = pd.DataFrame(data, columns=["Date", "Tenor", "MIBOR Rate (%)"])

# Convert "Date" to a Python datetime.date object
df["Date"] = pd.to_datetime(df["Date"], format="%d %b %Y").dt.date
df['MIBOR Rate (%)'] = df['MIBOR Rate (%)']/100
driver.quit()

In [ ]:
df.to_csv('./rf.csv')

In [ ]:
from arch import arch_model
import pandas as pd
import numpy as np
import math
from extract_options_data import ExtractOptionsData
from scipy.optimize import minimize, brentq
import scipy.optimize as opt
from scipy.stats import norm
import scipy.stats as si
import warnings
from scipy.interpolate import CubicSpline
from extract_options_chain import ExtractOptionsChain
from extract_options_data import ExtractOptionsData
from extract_rf import extract_risk_free_rate

warnings.filterwarnings('ignore')
import datetime

In [ ]:
DEFAULT_WINDOW = 30
DEFAULT_TRADING_PERIODS = 252
def close_to_close(daily_data, window=30, trading_periods=252, clean=True):
    """
    Calculate close-to-close volatility.

    :param window: Rolling window size for volatility calculation.
    :param trading_periods: Number of trading periods in a year for annualization.
    :param clean: If True, drop NaN values from the result.
    :return: Annualized close-to-close volatility.
    """
    if daily_data['close'].isnull().any():
        raise ValueError("Close prices contain NaN values.")

    log_return = (daily_data['close'] / daily_data['close'].shift(1)).apply(np.log)
    result = log_return.rolling(window=window, center=False).std() * math.sqrt(trading_periods)

    return result.dropna() if clean else result

def yang_zhang(daily_data, window=DEFAULT_WINDOW, trading_periods=DEFAULT_TRADING_PERIODS, clean=True):
    """
    Calculate volatility using the Yang-Zhang method.

    :param window: Rolling window size for volatility calculation.
    :param trading_periods: Number of trading periods in a year for annualization.
    :param clean: If True, drop NaN values from the result.
    :return: Annualized volatility using the Yang-Zhang method.
    """
    price_data = daily_data

    # Calculate log returns
    log_ho = (price_data['high'] / price_data['open']).apply(np.log)
    log_lo = (price_data['low'] / price_data['open']).apply(np.log)
    log_co = (price_data['close'] / price_data['open']).apply(np.log)
    log_oc = (price_data['open'] / price_data['close'].shift(1)).apply(np.log)
    log_oc_sq = log_oc ** 2
    log_cc = (price_data['close'] / price_data['close'].shift(1)).apply(np.log)
    log_cc_sq = log_cc ** 2

    # Calculate components
    rs = log_ho * (log_ho - log_co) + log_lo * (log_lo - log_co)
    close_vol = log_cc_sq.rolling(window=window, center=False).sum() * (1.0 / (window - 1.0))
    open_vol = log_oc_sq.rolling(window=window, center=False).sum() * (1.0 / (window - 1.0))
    window_rs = rs.rolling(window=window, center=False).sum() * (1.0 / (window - 1.0))

    # Combine components
    k = 0.34 / (1.34 + (window + 1) / (window - 1))
    result = (open_vol + k * close_vol + (1 - k) * window_rs).apply(np.sqrt) * math.sqrt(trading_periods)

    return result.dropna() if clean else result

def parkinson(daily_data, window=DEFAULT_WINDOW, trading_periods=DEFAULT_TRADING_PERIODS, clean=True):
    """
    Calculate volatility using the Parkinson method.

    :param window: Rolling window size for volatility calculation.
    :param trading_periods: Number of trading periods in a year for annualization.
    :param clean: If True, drop NaN values from the result.
    :return: Annualized volatility using the Parkinson method.
    """
    price_data = daily_data
    rs = (1.0 / (4.0 * math.log(2.0))) * ((price_data['high'] / price_data['low']).apply(np.log)) ** 2.0

    def f(v):
        return (trading_periods * v.mean()) ** 0.5

    result = rs.rolling(window=window, center=False).apply(func=f)
    return result.dropna() if clean else result

def garman_klass(daily_data, window=DEFAULT_WINDOW, trading_periods=DEFAULT_TRADING_PERIODS, clean=True):
    """
    Calculate volatility using the Garman-Klass method.

    :param window: Rolling window size for volatility calculation.
    :param trading_periods: Number of trading periods in a year for annualization.
    :param clean: If True, drop NaN values from the result.
    :return: Annualized volatility using the Garman-Klass method.
    """
    price_data = daily_data

    log_hl = (price_data['high'] / price_data['low']).apply(np.log)
    log_co = (price_data['close'] / price_data['open']).apply(np.log)
    rs = 0.5 * log_hl ** 2 - (2 * math.log(2) - 1) * log_co ** 2

    def f(v):
        return (trading_periods * v.mean()) ** 0.5

    result = rs.rolling(window=window, center=False).apply(func=f)
    return result.dropna() if clean else result

def rogers_satchell(daily_data, window=DEFAULT_WINDOW, trading_periods=DEFAULT_TRADING_PERIODS, clean=True):
    """
    Calculate volatility using the Rogers-Satchell method.

    :param window: Rolling window size for volatility calculation.
    :param trading_periods: Number of trading periods in a year for annualization.
    :param clean: If True, drop NaN values from the result.
    :return: Annualized volatility using the Rogers-Satchell method.
    """
    price_data = daily_data

    log_ho = (price_data['high'] / price_data['open']).apply(np.log)
    log_lo = (price_data['low'] / price_data['open']).apply(np.log)
    log_co = (price_data['close'] / price_data['open']).apply(np.log)
    rs = log_ho * (log_ho - log_co) + log_lo * (log_lo - log_co)

    def f(v):
        return (trading_periods * v.mean()) ** 0.5

    result = rs.rolling(window=window, center=False).apply(func=f)
    return result.dropna() if clean else result

def garch(daily_data, horizon=30, trading_periods=DEFAULT_TRADING_PERIODS):
    """
    Calculate volatility using a GARCH(1,1) model.

    :param horizon: Forecast horizon for GARCH model.
    :param trading_periods: Number of trading periods in a year for annualization.
    :return: Annualized GARCH volatility forecast.
    """
    return_series = (daily_data['close'] / daily_data['close'].shift(1)).apply(np.log).dropna()
    return_series = return_series * 100  # Scale for numerical stability

    model = arch_model(return_series, vol='GARCH', p=1, q=1, dist='t').fit(disp='off')
    forecast = model.forecast(horizon=horizon)
    # Extract the volatility for the last day of the horizon
    # This is the volatility over the entire horizon (e.g., 30 days)
    horizon_volatility = np.sqrt(forecast.variance.iloc[:, -1]) / 100  # Undo scaling

    # Convert horizon volatility to daily volatility
    daily_volatility = horizon_volatility  # / np.sqrt(horizon)

    # Annualize the daily volatility
    annualized_volatility = daily_volatility * np.sqrt(trading_periods)

    return annualized_volatility

def high_frequency(high_frequency_data, clean=True, trading_period=DEFAULT_TRADING_PERIODS, **kwargs):
    """
    Calculate volatility using high-frequency data.

    :param clean: If True, drop NaN values from the result.
    :param trading_period: Number of trading periods in a year for annualization.
    :param kwargs: Additional arguments for data extraction.
    :return: Annualized high-frequency volatility.
    """
    price_series = high_frequency_data
    price_series['Date'] = price_series.index.date
    price_series['prev_close'] = price_series['close'].shift(1)

    # Identify the first entry of each day
    first_of_day = price_series.groupby('Date').head(1).index

    # Compute log returns
    price_series['return'] = np.log(price_series['close'] / price_series['prev_close'])
    price_series.loc[first_of_day, 'return'] = np.log(
        price_series.loc[first_of_day, 'open'] / price_series.loc[first_of_day, 'prev_close'])

    return_series = price_series[['Date', 'return']].dropna()
    return_series['return'] = return_series['return'] ** 2

    # Compute daily variances and annualize
    annualized_daily_variance = return_series.groupby('Date')['return'].sum() * trading_period
    annualized_daily_volatility = annualized_daily_variance ** 0.5

    return annualized_daily_volatility.dropna() if clean else annualized_daily_volatility

def bsm_implied_volatility(mkt_price, S, K, rf, maturity, option_type, q=0, current_date=None):
    """
    Calculate the implied volatility using the Black-Scholes-Merton model.

    :param mkt_price: Market price of the option
    :param S: Current stock price
    :param K: Strike price
    :param r: Risk-free interest rate
    :param q: Dividend yield
    :param maturity: Maturity date of the option
    :param option_type: Type of option ('CE' for Call, 'PE' for Put)
    :param current_date: Current date (default is today)
    :return: Implied volatility or NaN if calculation fails
    """
    if current_date is None:
        current_date = datetime.date.today()

    # Calculate time to maturity in trading days and calendar days
    trading_days = np.busday_count(current_date, maturity)
    calendar_days = np.busday_count(current_date, maturity, weekmask='1111111')

    r = np.mean(rf['MIBOR Rate (%)'])

    # if calendar_days < 14:
    #     r = rf[rf['Tenor'] == 14]['MIBOR Rate (%)'].values[0]
    # else:
    #     x = np.array(rf['Tenor'])
    #     y = np.array(rf['MIBOR Rate (%)'])
    #     cs = CubicSpline(x, y)
    #     r = cs(np.array(calendar_days))

    t1 = trading_days / 252  # Trading days to years
    t2 = calendar_days / 365  # Calendar days to years

    if t1 <= 0 or t2 <= 0:
        return np.nan

    def bsm_price(sigma):
        """Calculate the Black-Scholes-Merton option price."""
        d1 = (np.log(S / K) + (r - q) * t2 + 0.5 * sigma ** 2 * t1) / (sigma * np.sqrt(t1))
        d2 = d1 - sigma * np.sqrt(t1)

        if option_type == 'CE':
            return S * np.exp(-q * t2) * si.norm.cdf(d1) - K * np.exp(-r * t2) * si.norm.cdf(d2)
        elif option_type == 'PE':
            return K * np.exp(-r * t2) * si.norm.cdf(-d2) - S * np.exp(-q * t2) * si.norm.cdf(-d1)
        else:
            raise ValueError("Invalid option type. Use 'CE' for Call or 'PE' for Put.")

    def objective_function(sigma):
        """Objective function for root-finding."""
        return bsm_price(sigma) - mkt_price

    try:
        implied_vol = brentq(objective_function, 0, 5)
        return implied_vol
    except ValueError as e:
        print(f"Failed to find implied volatility: {e}")
        return np.nan

def corrado_su_implied_moments(mkt_price, S, K, rf, maturity, option_type, q=0, current_date=None):
    """
    Compute the implied moments (volatility, skew, and kurtosis) for an option using the Corrado-Su adjustment.

    Parameters:
        mkt_price : array-like
            Market prices of the option.
        S : float
            Underlying asset price.
        K : array-like
            Strike prices.
        rf : DataFrame
            Risk-free rate data containing 'MIBOR Rate (%)'.
        maturity : datetime.date
            Maturity date of the option.
        option_type : str
            Option type: 'CE' for call or 'PE' for put.
        q : float, optional
            Dividend yield (default is 0).
        current_date : datetime.date, optional
            Current date (default is today's date).

    Returns:
        pd.DataFrame or np.nan:
            DataFrame with the implied volatility, skew, and kurtosis if successful; otherwise, NaN.
    """

    # Compute risk-free rate as the average of MIBOR rates.
    r = np.mean(rf['MIBOR Rate (%)'])

    # Use today's date if current_date is not provided.
    if current_date is None:
        current_date = datetime.date.today()

    # Calculate time to maturity:
    # trading_days: business days between current_date and maturity.
    trading_days = np.busday_count(current_date, maturity)
    # calendar_days: calendar days between current_date and maturity (using all days of the week).
    calendar_days = np.busday_count(current_date, maturity, weekmask='1111111')

    # Convert trading days and calendar days to year fractions.
    t1 = trading_days / 252  # Trading days to years.
    t2 = calendar_days / 365  # Calendar days to years.

    # If the computed time to maturity is zero or negative, return NaN.
    if t1 <= 0 or t2 <= 0:
        return np.nan

    def bsm_price(k, sigma):
        """
        Calculate the Black-Scholes-Merton option price for a given strike (k) and volatility (sigma).

        Parameters:
            k : float
                Strike price.
            sigma : float
                Volatility.

        Returns:
            float:
                Option price computed using the BSM formula.
        """
        d1 = (np.log(S / k) + (r - q) * t2 + 0.5 * sigma ** 2 * t1) / (sigma * np.sqrt(t1))
        d2 = d1 - sigma * np.sqrt(t1)

        # Return price based on option type.
        if option_type == 'CE':
            # European Call Option.
            return S * np.exp(-q * t2) * si.norm.cdf(d1) - k * np.exp(-r * t2) * si.norm.cdf(d2)
        elif option_type == 'PE':
            # European Put Option.
            return k * np.exp(-r * t2) * si.norm.cdf(-d2) - S * np.exp(-q * t2) * si.norm.cdf(-d1)
        else:
            raise ValueError("Invalid option type. Use 'CE' for Call or 'PE' for Put.")

    def gram_charlier_adjustment(k, sigma, skew, kurt):
        """
        Compute the Gram-Charlier adjustment terms for skewness and kurtosis.

        Parameters:
            k : float
                Strike price.
            sigma : float
                Volatility.
            skew : float
                Skewness parameter.
            kurt : float
                Kurtosis parameter.

        Returns:
            tuple:
                Q3 and Q4 adjustment terms.
        """
        d1 = (np.log(S / k) + (r - q) * t2 + 0.5 * sigma ** 2 * t1) / (sigma * np.sqrt(t1))
        d2 = d1 - sigma * np.sqrt(t1)

        # Compute Q3 and Q4 using the standard normal density function.
        term_1 = S * sigma * np.sqrt(t1)
        term_2 = (2 * sigma * np.sqrt(t1) - d1) * si.norm.pdf(d1)
        term_3 = si.norm.cdf(d1) * (sigma ** 2)

        Q3 = (1 / 6) * term_1 * (term_2 + term_3)

        term_4 = (d1 ** 2) - 1 - (3 * sigma * np.sqrt(t1) * d2) * si.norm.pdf(d1)
        term_5 = (sigma ** 3) * np.sqrt(t1 ** 3) * si.norm.cdf(d1)
        Q4 = (1 / 24) * term_1 * (term_4 + term_5)
        return Q3, Q4

    def corrado_su_price(k, sigma, skew, kurt):
        """
        Calculate the option price using the Corrado-Su model, which adjusts the BSM price with skew and kurtosis corrections.

        Parameters:
            k : float
                Strike price.
            sigma : float
                Volatility.
            skew : float
                Skewness parameter.
            kurt : float
                Kurtosis parameter.

        Returns:
            float:
                Adjusted option price.
        """
        # Base price from the Black-Scholes-Merton model.
        BSM_PRICE = bsm_price(k, sigma)
        # Compute adjustment terms.
        Q3, Q4 = gram_charlier_adjustment(k, sigma, skew, kurt)
        # Adjust the BSM price with skew and kurtosis corrections.
        cs_price = BSM_PRICE + skew * Q3 + (kurt - 3) * Q4
        return cs_price

    def objective(params, K):
        """
        Objective function for optimization: minimize the sum of squared differences between
        the model prices (using Corrado-Su adjustments) and the market prices.

        Parameters:
            params : list or array-like
                Parameters to optimize: [sigma, skew, kurt].
            K : array-like
                Strike prices.

        Returns:
            float:
                Sum of squared errors.
        """
        sigma, skew, kurt = params
        # Compute the model price for each strike.
        model_prices = np.array([corrado_su_price(k, sigma, skew, kurt) for k in K])
        return np.sum((model_prices - mkt_price) ** 2)

    # Initial guess for the parameters: volatility, skew, kurtosis.
    initial_guess = [0.1, 0, 3]

    try:
        # Optimize the parameters to best fit the market prices.
        result = opt.minimize(objective, initial_guess, args=(K,), bounds=[(0.01, 1), (-2, 2), (1, 10)])
    except Exception as e:
        # If an exception occurs during optimization, print the error and return NaN.
        print(f"Optimization failed: {e}")
        return np.nan

    # If optimization was unsuccessful, notify and return NaN.
    if not result.success:
        print("Optimization did not converge.")
        return np.nan

    # Extract optimal parameters: implied volatility, skew, and kurtosis.
    implied_vol, implied_skew, implied_kurt = result.x

    # Create and return a DataFrame with the implied moments.
    df = pd.DataFrame({
        'cs_implied_vol': [implied_vol],
        'cs_implied_skew': [implied_skew],
        'cs_implied_kurt': [implied_kurt]
    })
    return df

In [ ]:
options_json = {}
symbols_json = {}
symbols_json['underlying'] = 'NIFTY'
symbols_json['type'] = 'index'
realized_vol_json = {
            'Close-to-close': close_to_close(daily_data),
            'Yang-Zhang': yang_zhang(daily_data),
            'Parkinson': parkinson(daily_data),
            'Garmann_Klass': garman_klass(daily_data),
            'Rogers_Satchell': rogers_satchell(daily_data),
            'High_Frequency': high_frequency(high_frequency_data),
            'GARCH(1,1)': garch(daily_data)
        }

In [ ]:
means = [s.mean() for s in realized_vol_json.values()]
forecasted_realized_volatility = sum(means) / len(means)
symbols_json['realized_volatility'] = realized_vol_json

In [ ]:
from greeks import Extract_Greeks

In [ ]:
rf = df

In [ ]:
ce_json = {}
pe_json = {}
for expiry in call_df['expiryDate'].unique():
    expiry_json = {}
    implied_moments_json = {}
    greeks_json = {}
    # Use single bracket filtering (not double brackets)
    ce_chain_df = call_df[call_df['expiryDate'] == expiry]
    # print(ce_chain_df)
    try:
        implied_moments_json['bsm_implied_vol'] = bsm_implied_volatility(
            mkt_price=ce_chain_df['mkt_price'].values[0],
            S=ce_chain_df['ltp'].values[0],
            K=ce_chain_df['strikePrice'].values[0],
            rf=rf, maturity=expiry,
            option_type='CE', q=0, current_date=None
        )
        cs_moments = corrado_su_implied_moments(
            mkt_price=np.array(ce_chain_df['mkt_price']),
            S=ce_chain_df['ltp'].values[0],
            K=np.array(ce_chain_df['strikePrice']),
            rf=rf, maturity=expiry, option_type='CE', q=0,
            current_date=None
        )
        implied_moments_json['cs_implied_vol'] = cs_moments['cs_implied_vol'].values[0]
        implied_moments_json['cs_implied_skew'] = cs_moments['cs_implied_skew'].values[0]
        implied_moments_json['cs_implied_kurt'] = cs_moments['cs_implied_kurt'].values[0]
        implied_moments_json['edge'] = np.abs(
            implied_moments_json[
                'bsm_implied_vol'] - forecasted_realized_volatility) / forecasted_realized_volatility
        # print(expiry_json)
    except Exception as e:
        print(f"❌ Error processing call option for expiry {expiry}: {e}")

    object_greeks = Extract_Greeks(S=ce_chain_df['ltp'].values[0],
                                   K=ce_chain_df['strikePrice'].values[0],
                                   imp_vol = implied_moments_json['bsm_implied_vol'],
                                   rf=rf, maturity=expiry, option_type='CE', q=0, current_date=None)
    greeks_json['delta'] = object_greeks.delta()
    greeks_json['gamma'] = object_greeks.gamma()
    greeks_json['vega'] = object_greeks.vega()
    greeks_json['rho'] = object_greeks.rho()
    greeks_json['theta'] = object_greeks.theta()

    expiry_json['implied_moments'] = implied_moments_json
    expiry_json['greeks'] = greeks_json
    ce_json[expiry] = expiry_json

In [ ]:
for expiry in put_df['expiryDate'].unique():
    expiry_json = {}
    implied_moments_json = {}
    greeks_json = {}
    pe_chain_df = put_df[put_df['expiryDate'] == expiry]
    try:
        implied_moments_json['bsm_implied_vol'] = bsm_implied_volatility(
            mkt_price=pe_chain_df['mkt_price'].values[0],
            S=pe_chain_df['ltp'].values[0],
            K=pe_chain_df['strikePrice'].values[0],
            rf=rf, maturity=expiry,
            option_type='PE', q=0, current_date=None
        )
        cs_moments = corrado_su_implied_moments(
            mkt_price=np.array(pe_chain_df['mkt_price']),
            S=pe_chain_df['ltp'].values[0],
            K=np.array(pe_chain_df['strikePrice']),
            rf=rf, maturity=expiry, option_type='PE', q=0,
            current_date=None
        )
        implied_moments_json['cs_implied_vol'] = cs_moments['cs_implied_vol'].values[0]
        implied_moments_json['cs_implied_skew'] = cs_moments['cs_implied_skew'].values[0]
        implied_moments_json['cs_implied_kurt'] = cs_moments['cs_implied_kurt'].values[0]
        implied_moments_json['edge'] = np.abs(
            implied_moments_json[
                'bsm_implied_vol'] - forecasted_realized_volatility) / forecasted_realized_volatility
        # print(expiry_json)
    except Exception as e:
        print(f"❌ Error processing put option for expiry {expiry}: {e}")
    object_greeks = Extract_Greeks(S=pe_chain_df['ltp'].values[0],
                                   K=pe_chain_df['strikePrice'].values[0],
                                   imp_vol = implied_moments_json['bsm_implied_vol'],
                                   rf=rf, maturity=expiry, option_type='PE', q=0, current_date=None)
    greeks_json['delta'] = object_greeks.delta()
    greeks_json['gamma'] = object_greeks.gamma()
    greeks_json['vega'] = object_greeks.vega()
    greeks_json['rho'] = object_greeks.rho()
    greeks_json['theta'] = object_greeks.theta()

    expiry_json['implied_moments'] = implied_moments_json
    expiry_json['greeks'] = greeks_json

    pe_json[expiry] = expiry_json
    # print(pe_json)

symbols_json['CE'] = ce_json
symbols_json['PE'] = pe_json
# print(symbols_json)

options_json['^NSEI'] = symbols_json

In [ ]:
options_json

In [ ]:
# Save the dictionary to a pickle file
with open("options_dict.pkl", "wb") as file:
    pickle.dump(options_json, file)

In [ ]:
import plotly.express as px

In [ ]:
options_json['^NSEI']['realized_volatility'].keys()

In [ ]:
df = options_json['^NSEI']['realized_volatility']['Close-to-close']
fig = px.line(df)
fig.show()

# Testing options_strategies.py 

In [2]:
call_chain = pd.read_csv(
    './calls.csv',
    index_col='Unnamed: 0',
    converters={'expiryDate': lambda x: pd.to_datetime(x).date()}
)
put_chain = pd.read_csv(
    './puts.csv',
    index_col='Unnamed: 0',
    converters={'expiryDate': lambda x: pd.to_datetime(x).date()}
)

In [3]:
call_chain

,symbol,strikePrice,expiryDate,bid_qty,bid_price,ask_qty,ask_price,ltp,S-K,calendar_days_to_maturity,mkt_price
302,NIFTY,22550,2025-04-24,75,571.95,75,574.35,22571.65,21.65,48,573.150
301,NIFTY,22550,2025-04-09,1800,451.45,75,493.90,22571.65,21.65,33,472.675
308,NIFTY,22600,2025-04-09,900,411.60,75,451.10,22571.65,28.35,33,431.350
309,NIFTY,22600,2025-04-24,75,541.75,150,543.75,22571.65,28.35,48,542.750
294,NIFTY,22500,2025-04-09,75,483.25,75,504.95,22571.65,71.65,33,494.100
...,...,...,...,...,...,...,...,...,...,...,...
641,NIFTY,25300,2025-04-24,225,10.15,225,12.80,22571.65,2728.35,48,11.475
644,NIFTY,25350,2025-04-24,225,9.15,225,13.05,22571.65,2778.35,48,11.100
646,NIFTY,25400,2025-04-24,75,10.00,1050,15.25,22571.65,2828.35,48,12.625
648,NIFTY,25450,2025-04-24,225,9.00,600,14.45,22571.65,2878.35,48,11.725


In [4]:
itm_calls = call_chain[call_chain['ltp'] - call_chain['strikePrice'] > 0]
itm_calls

,symbol,strikePrice,expiryDate,bid_qty,bid_price,ask_qty,ask_price,ltp,S-K,calendar_days_to_maturity,mkt_price
302,NIFTY,22550,2025-04-24,75,571.95,75,574.35,22571.65,21.65,48,573.150
301,NIFTY,22550,2025-04-09,1800,451.45,75,493.90,22571.65,21.65,33,472.675
294,NIFTY,22500,2025-04-09,75,483.25,75,504.95,22571.65,71.65,33,494.100
295,NIFTY,22500,2025-04-24,300,605.25,150,607.20,22571.65,71.65,48,606.225
287,NIFTY,22450,2025-04-09,900,511.55,1350,559.65,22571.65,121.65,33,535.600
288,NIFTY,22450,2025-04-24,75,633.55,75,646.35,22571.65,121.65,48,639.950
281,NIFTY,22400,2025-04-24,75,671.30,75,674.15,22571.65,171.65,48,672.725
280,NIFTY,22400,2025-04-09,900,534.20,75,582.65,22571.65,171.65,33,558.425
274,NIFTY,22350,2025-04-24,75,703.90,75,707.30,22571.65,221.65,48,705.600
273,NIFTY,22350,2025-04-09,1800,577.30,1800,627.15,22571.65,221.65,33,602.225


In [5]:
with open("symbols_json.pkl", "rb") as file:
    symbols_json = pickle.load(file)

In [6]:
symbols_json

{'^NSEI': {'underlying': 'NIFTY',
  'type': 'index',
  'realized_volatility': {'Close-to-close': date
   2020-04-27                   0.754585
   2020-04-28                   0.755656
   2020-04-29                   0.715575
   2020-04-30                   0.713138
   2020-05-04                   0.696063
                                  ...   
   2025-03-03                   0.120557
   2025-03-04                   0.120306
   2025-03-05                   0.118085
   2025-03-06                   0.118569
   2025-03-07 14:00:23+05:30    0.120754
   Name: close, Length: 1206, dtype: float64,
   'Yang-Zhang': date
   2020-04-27                   0.766293
   2020-04-28                   0.766498
   2020-04-29                   0.749484
   2020-04-30                   0.674533
   2020-05-04                   0.669401
                                  ...   
   2025-03-03                   0.125386
   2025-03-04                   0.126366
   2025-03-05                   0.126056
   2025-03

In [7]:
#ops_data_obj = ExtractOptionsData()
q = 0
rf = pd.read_csv('./rf.csv')

In [8]:
call_chain['expiryDate'].unique()[0]

datetime.date(2025, 4, 24)

In [22]:
strategies_json = {}
for expiry in call_chain['expiryDate'].unique():
    ce_chain_df = call_chain[call_chain['expiryDate'] == expiry]
    pe_chain_df = put_chain[put_chain['expiryDate'] == expiry]
    expiry_strategy_json = {}
    #Strategies
    strategies = Strategies(expiry=expiry, ce_chain=ce_chain_df, pe_chain=pe_chain_df, rf=rf, q=q )
    expiry_strategy_json['Long Call'] = strategies.long_call()
    expiry_strategy_json['Long Put'] = strategies.long_put()
    expiry_strategy_json['Short Call'] = strategies.short_call()
    expiry_strategy_json['Short Put'] = strategies.short_put()
    expiry_strategy_json['Bull Call Spread'] = strategies.bull_call_spread()
    expiry_strategy_json['Bull Put Spread'] = strategies.bull_put_spread()
    expiry_strategy_json['Bear Call Spread'] = strategies.bear_call_spread()
    expiry_strategy_json['Bear Put Spread'] = strategies.bear_put_spread()
    expiry_strategy_json['Long Call Butterfly'] = strategies.long_call_butterfly()
    expiry_strategy_json['Long Put Butterfly'] = strategies.long_put_butterfly()
    print("Long Straddle")
    expiry_strategy_json['Long Straddle'] = strategies.long_straddle()
    print("Short Straddle")
    expiry_strategy_json['Short Straddle'] = strategies.short_straddle()
    print("Strip")
    expiry_strategy_json['Strip'] = strategies.strip()
    print("Strap")
    expiry_strategy_json['Strap'] = strategies.strap()
    print("Long Strangle")
    expiry_strategy_json['Long Strangle'] = strategies.long_strangle()
    print("Short Strangle")
    expiry_strategy_json['Short Strangle'] = strategies.short_strangle()
    strategies_json[expiry] = expiry_strategy_json


transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0


transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0


transaction cost= 0.004575657706780853 	spread= 0.0039843577067808535

transaction cost= 0.004575657706780853 	spread= 0.0039843577067808535

transaction cost= 0.004575657706780853 	spread= 0.0039843577067808535

transaction cost= 0.004575657706780853 	spread= 0.0039843577067808535

transaction cost= 0.004575657706780853 	spread= 0.0039843577067808535

transaction cost= 0.004575657706780853 	spread= 0.0039843577067808535

transaction cost= 0.004575657706780853 	spread= 0.0039843577067808535

transaction cost= 0.004575657706780853 	spread= 0.0039843577067808535

transaction cost= 0.004575657706780853 	spread= 0.0039843577067808535

transaction cost= 0.004575657706780853 	spread= 0.0039843577067808535

transaction cost= 0.004575657706780853 	spread= 0.0039843577067808535

transaction cost= 0.004575657706780853 	spread= 0.0039843577067808535

transaction cost= 0.004575657706780853 	spread= 0.0039843577067808535

transaction cost= 0.004575657706780853 	spread= 0.0039843577067808535

trans


transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004575657706780853 	spread= 0.0039843577067808535

transaction cost= 0.004575657706780853 	spread= 0.0039843577067808535

transaction cost= 0.004575657706780853 	spread= 0.0039843577067808535

transaction cost= 0.004575657706780853 	spread= 0.0039843577067808535

transaction cos


transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0.004778685501177662 	spread= 0.004187385501177663

transaction cost= 0

...


transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transacti


transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transacti


transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transacti


transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transacti


transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transaction cost= 0.018591299999999998 	spread= 0.018

transacti

In [19]:
put_chain

,symbol,strikePrice,expiryDate,bid_qty,bid_price,ask_qty,ask_price,ltp,K-S,calendar_days_to_maturity,mkt_price
388,NIFTY,22550,2025-04-09,1800,250.05,1800,295.35,22571.65,21.65,33,272.700
389,NIFTY,22550,2025-04-24,75,319.85,225,321.65,22571.65,21.65,48,320.750
395,NIFTY,22600,2025-04-09,0,0.00,900,314.85,22571.65,28.35,33,157.425
396,NIFTY,22600,2025-04-24,75,338.15,150,339.50,22571.65,28.35,48,338.825
382,NIFTY,22500,2025-04-24,75,301.75,300,302.40,22571.65,71.65,48,302.075
...,...,...,...,...,...,...,...,...,...,...,...
715,NIFTY,25300,2025-04-24,75,2420.40,75,2536.50,22571.65,2728.35,48,2478.450
718,NIFTY,25350,2025-04-24,225,2467.80,75,2586.60,22571.65,2778.35,48,2527.200
720,NIFTY,25400,2025-04-24,225,2486.80,225,2644.75,22571.65,2828.35,48,2565.775
722,NIFTY,25450,2025-04-24,225,2556.90,225,2701.05,22571.65,2878.35,48,2628.975


In [ ]:
symbols_json

In [ ]:
# Save the dictionary to a pickle file
with open("symbols_json.pkl", "wb") as file:
    pickle.dump(symbols_json, file)

### Volatility Cones

In [ ]:
extract_data_object = ExtractOptionsData()
cones_data = extract_data_object.extracting_ohlc(ticker='NIFTY', category='index', period='5y',
                                                              interval='1d')

In [ ]:
cones_df = pd.DataFrame()
cones_df['ln_returns'] = (cones_data['close'] / cones_data['close'].shift(1)).apply(np.log)

In [ ]:
cones_df.index = cones_df.index.map(lambda x: x.date() if hasattr(x, 'date') else x)

In [ ]:
cones_df.dropna(inplace=True)

In [ ]:
def func_m(window, size_return_series):
    # Ensure that there is enough data for the given window size.
    if size_return_series < window:
        raise ValueError(f"Not enough data points ({size_return_series}) for the window size {window}.")
    n = size_return_series - window + 1
    if n <= 0:
        raise ValueError(f"Computed number of periods ({n}) is invalid. Check window size and data length.")
    t1 = window / n
    t2 = (window ** 2) - 1
    t3 = 3 * (n ** 2)
    denominator = 1 - t1 + (t2 / t3)
    if denominator <= 0:
        raise ValueError("Denominator in scaling factor calculation is non-positive. Check inputs.")
    m = (1 / denominator) ** 0.5
    return m

In [ ]:
windows = [20, 40, 60, 120, 240]
for window in windows:
    # Calculate the rolling standard deviation of the logarithmic returns.
    rolling_std = cones_df['ln_returns'].rolling(window=window).std()

    # Ensure DEFAULT_TRADING_PERIODS is defined.
    try:
        scaling = math.sqrt(252)
    except NameError:
        raise NameError("DEFAULT_TRADING_PERIODS is not defined.")

    # Compute the scaling factor for the current window.
    try:
        m_value = func_m(window, len(cones_df['ln_returns']))
    except ValueError as ve:
        raise ValueError(f"Error computing scaling factor for window {window}: {ve}")

    # Compute the volatility and store it in a new column.
    cones_df[f'{window}_day_vol'] = rolling_std * scaling * m_value

# Remove any rows with NaN values resulting from the rolling window calculations.
cones_df.dropna(inplace=True)

In [ ]:
cones_df

In [ ]:
symbols_json['^NSEI']['realized_volatility']['volatility_cones_df'] = cones_df

In [ ]:
windows = [20, 40, 60, 120, 240]
skewness_cones_df = pd.DataFrame()
for window in windows:
    # Calculate the rolling skewness of the logarithmic returns.
    rolling_skew = cones_df['ln_returns'].rolling(window=window).skew()

    # Annual scaling for skewness is 1 / sqrt(trading days).
    try:
        trading_days = DEFAULT_TRADING_PERIODS  # DEFAULT_TRADING_PERIODS should be defined (typically 252)
    except NameError:
        trading_days = 252

    scaling = 1 / math.sqrt(trading_days)

    # Compute the annualized skewness and store it in a new column.
    skewness_cones_df[f'{window}_day_skew'] = rolling_skew * scaling 

# Remove any rows with NaN values resulting from the rolling window calculations.
skewness_cones_df.dropna(inplace=True)

In [ ]:
symbols_json['^NSEI']['realized_volatility']['skewness_cones_df'] = skewness_cones_df

In [ ]:
windows = [20, 40, 60, 120, 240]
kurtosis_cones_df = pd.DataFrame()
for window in windows:
    # Calculate the rolling excess kurtosis of the logarithmic returns.
    rolling_excess_kurt = cones_df['ln_returns'].rolling(window=window).kurt()

    # Annual scaling for excess kurtosis is 1 / trading days.
    try:
        trading_days = DEFAULT_TRADING_PERIODS  # DEFAULT_TRADING_PERIODS should be defined (typically 252)
    except NameError:
        trading_days = 252

    scaling = 1 / trading_days

    # Compute the annualized excess kurtosis and store it in a new column.
    kurtosis_cones_df[f'{window}_day_excess_kurt'] = rolling_excess_kurt * scaling 

# Remove any rows with NaN values resulting from the rolling window calculations.
kurtosis_cones_df.dropna(inplace=True)

In [ ]:
symbols_json['^NSEI']['realized_volatility']['kurtosis_cones_df'] = kurtosis_cones_df

In [ ]:
kurtosis_cones_df

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

def plot_volatility_cone(cones_df, windows=[20, 40, 60, 120, 240], quantiles=[0.25, 0.75]):
    """
    Plots a volatility cone figure from a precomputed cones_df DataFrame.

    Parameters:
        cones_df : pd.DataFrame
            DataFrame containing volatility series for each rolling window.
            Expected columns: '20_day_vol', '40_day_vol', etc.
        windows : list of int
            Rolling window sizes (in days) to plot.
        quantiles : list of float
            Two-element list containing the lower and upper quantiles 
            (e.g., [0.25, 0.75]) used for the cone boundaries.

    Returns:
        fig : matplotlib.figure.Figure
            The generated matplotlib figure.
    """
    # Check that quantiles are correctly provided.
    if len(quantiles) != 2:
        raise ValueError("A two-element list for quantiles is required (e.g., [0.25, 0.75]).")
    if quantiles[0] >= quantiles[1]:
        raise ValueError("The first quantile must be less than the second.")
    
    # Prepare lists to store summary statistics.
    max_vals = []
    min_vals = []
    top_quantiles = []
    medians = []
    bottom_quantiles = []
    realized = []
    boxplot_data = []  # For the boxplot later
    
    # Loop over each window size and extract the corresponding column.
    for window in windows:
        col_name = f"{window}_day_vol"
        if col_name not in cones_df.columns:
            raise ValueError(f"Expected column '{col_name}' not found in DataFrame.")
        
        # Drop NaNs from the series for accurate statistics.
        vol_series = cones_df[col_name].dropna()
        
        # Compute the summary statistics.
        max_vals.append(vol_series.max())
        min_vals.append(vol_series.min())
        top_quantiles.append(vol_series.quantile(quantiles[1]))
        medians.append(vol_series.median())
        bottom_quantiles.append(vol_series.quantile(quantiles[0]))
        
        # Save the series for boxplot.
        boxplot_data.append(vol_series)
    
    # Create the figure with two axes: one for the line plot (cone) and one for the boxplot.
    fig = plt.figure(figsize=(10, 6))
    # Define layout parameters (mimicking the original design)
    left, width = 0.07, 0.65
    bottom, height = 0.2, 0.7
    left_box = left + width + 0.02
    rect_cones = [left, bottom, width, height]
    rect_box = [left_box, bottom, 0.17, height]
    
    ax_cones = plt.axes(rect_cones)
    ax_box = plt.axes(rect_box)
    
    # Plot the volatility cone as line plots.
    ax_cones.plot(windows, max_vals, label="Max", marker='o')
    ax_cones.plot(windows, top_quantiles, label=f"{int(quantiles[1]*100)} Prctl", marker='o')
    ax_cones.plot(windows, medians, label="Median", marker='o')
    ax_cones.plot(windows, bottom_quantiles, label=f"{int(quantiles[0]*100)} Prctl", marker='o')
    ax_cones.plot(windows, min_vals, label="Min", marker='o')

    
    # Set x-axis ticks and limits.
    ax_cones.set_xticks(windows)
    ax_cones.set_xlim((windows[0] - 5, windows[-1] + 5))
    
    # Format the y-axis labels.
    # Here we assume the volatility is expressed as a fraction (e.g., 0.02 for 2%).
    def format_y(y):
        return f"{round(y*100, 0)}%"
    ytick_labels = [format_y(y) for y in ax_cones.get_yticks()]
    ax_cones.set_yticklabels(ytick_labels)
    
    ax_cones.grid(True, axis='y', which='major', alpha=0.5)
    ax_cones.set_title("Volatility Cone")
    ax_cones.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=3)
    
    # Create a boxplot on the right side.
    ax_box.violinplot(boxplot_data, showmeans=True, quantiles=[[0.05,0.95],[0.05,0.95],[0.05,0.95],[0.05,0.95],[0.05,0.95]])
    #ax_box.boxplot(boxplot_data, notch=True, sym='+', patch_artist=True)
    
    # Format the y-axis for the boxplot.
    ytick_labels_box = [format_y(y) for y in ax_box.get_yticks()]
    ax_box.set_yticklabels(ytick_labels_box)
    ax_box.yaxis.tick_right()
    #ax_box.set_xticks(windows)
    ax_box.grid(True, axis='y', which='major', alpha=0.5)
    
    plt.show()
    return fig

In [ ]:
fig = plot_volatility_cone(cones_df)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

def plot_volatility_cone_plotly(cones_df, windows=[20, 40, 60, 120, 240], quantiles=[0.25, 0.75]):
    """
    Plots a volatility cone figure using Plotly.
    
    Parameters:
        cones_df : pd.DataFrame
            DataFrame containing volatility series for each rolling window.
            Expected columns: '20_day_vol', '40_day_vol', etc.
        windows : list of int
            Rolling window sizes (in days) to plot.
        quantiles : list of float
            Two-element list containing the lower and upper quantiles (e.g., [0.25, 0.75])
            used for the cone boundaries.
            
    Returns:
        fig : plotly.graph_objects.Figure
            The generated Plotly figure.
    """
    # Check that quantiles are correctly provided.
    if len(quantiles) != 2:
        raise ValueError("A two-element list for quantiles is required (e.g., [0.25, 0.75]).")
    if quantiles[0] >= quantiles[1]:
        raise ValueError("The first quantile must be less than the second.")
    
    # Prepare lists to store summary statistics and data for violin plots.
    max_vals = []
    min_vals = []
    top_quantiles = []
    medians = []
    bottom_quantiles = []
    violin_data = []  # For the violin plot
    
    for window in windows:
        col_name = f"{window}_day_vol"
        if col_name not in cones_df.columns:
            raise ValueError(f"Expected column '{col_name}' not found in DataFrame.")
        
        # Drop NaNs from the series for accurate statistics.
        vol_series = cones_df[col_name].dropna()
        
        # Compute the summary statistics.
        max_vals.append(vol_series.max())
        min_vals.append(vol_series.min())
        top_quantiles.append(vol_series.quantile(quantiles[1]))
        medians.append(vol_series.median())
        bottom_quantiles.append(vol_series.quantile(quantiles[0]))
        
        violin_data.append(vol_series)
    
    # Create a subplot with 1 row and 2 columns.
    # Adjust column widths to mimic the original layout (e.g., wider left panel).
    fig = make_subplots(rows=1, cols=2,
                        column_widths=[0.75, 0.25],
                        subplot_titles=("Volatility Cone", "Distribution"))
    
    # Add the volatility cone line plots to the left subplot.
    fig.add_trace(go.Scatter(x=windows, y=max_vals, mode='lines+markers',
                             name='Max'), row=1, col=1)
    fig.add_trace(go.Scatter(x=windows, y=top_quantiles, mode='lines+markers',
                             name=f"{int(quantiles[1]*100)} Prctl"), row=1, col=1)
    fig.add_trace(go.Scatter(x=windows, y=medians, mode='lines+markers',
                             name='Median'), row=1, col=1)
    fig.add_trace(go.Scatter(x=windows, y=bottom_quantiles, mode='lines+markers',
                             name=f"{int(quantiles[0]*100)} Prctl"), row=1, col=1)
    fig.add_trace(go.Scatter(x=windows, y=min_vals, mode='lines+markers',
                             name='Min'), row=1, col=1)
    
    # Format the left subplot (volatility cone):
    # Format y-axis as percentages (e.g., 2% for 0.02).
    fig.update_yaxes(tickformat=',.0%', row=1, col=1)
    # Set custom x-axis ticks and limits.
    fig.update_xaxes(tickmode='array', tickvals=windows,
                     range=[windows[0] - 5, windows[-1] + 5], row=1, col=1)
    
    # Add violin plots to the right subplot for each rolling window.
    # Each trace corresponds to one window; using the window size as a categorical x value.
    for i, window in enumerate(windows):
        fig.add_trace(go.Violin(y=violin_data[i],
                                x=[str(window)] * len(violin_data[i]),
                                name=f"{window}-day",
                                box_visible=True,
                                meanline_visible=True,
                                showlegend=False),
                      row=1, col=2)
    
    # Format the right subplot's y-axis similarly.
    fig.update_yaxes(tickformat=',.0%', row=1, col=2)
    
    # Update the layout: adjust margins and position the legend at the bottom center.
     # Update the layout with a white background.
    fig.update_layout(
        template="plotly_white",  # Sets both paper and plot background to white.
        legend=dict(orientation="h", yanchor="bottom", y=-0.2, xanchor="center", x=0.5),
        margin=dict(l=50, r=50, t=50, b=50)
    )
    
    fig.show()
    return fig

# Example usage:
# Assuming 'cones_df' is your DataFrame with the required columns.
# fig = plot_volatility_cone_plotly(cones_df)

In [ ]:
fig = plot_volatility_cone_plotly(cones_df)

### Plotting Strategies

In [11]:
symbols_json['strategies'][datetime.date(2025, 4, 24)]

{'Long Call': {'greeks': {'delta': 0.5568569282562195,
   'gamma': 0.00031534684601563156,
   'vega': 2511.0036223400402,
   'rho': 920.2433730709204,
   'theta': -3965.1526484431006,
   'zak_lower_band': 0.5455383835789277,
   'zak_upper_band': 0.55268321223714},
  'payoff': {'payoffs':          payoff     delta         gamma       vega  zak_lower_band  \
   0   -574.350000  0.000069  2.793769e-07   1.434698        0.000194   
   1   -574.350000  0.000099  3.887876e-07   2.016391        0.000319   
   2   -574.350000  0.000140  5.359301e-07   2.806995        0.000493   
   3   -574.350000  0.000197  7.318715e-07   3.870959        0.000732   
   4   -574.350000  0.000274  9.902585e-07   5.288857        0.001052   
   ..          ...       ...           ...        ...             ...   
   95  3533.054644  0.999161  1.938106e-06  21.525110        0.996035   
   96  3622.851449  0.999319  1.593698e-06  17.819477        0.996684   
   97  3712.648254  0.999449  1.306567e-06  14.707260    

In [18]:
df = symbols_json['strategies'][datetime.date(2025, 4, 24)]['Bull Put Spread']['payoff']['payoffs']

In [21]:
strategies_json

{datetime.date(2025, 4, 24): {'Short Strangle': {'greeks': {'delta': -0.10920007664263998,
    'gamma': -0.0007206245626743565,
    'vega': -4883.932297471161,
    'rho': -122.13808101006191,
    'theta': 5906.731853259997,
    'zak_lower_band': -0.10135389469093031,
    'zak_upper_band': -0.11704625859434964},
   'payoff': {'payoffs':                S       payoff     delta         gamma       vega  \
    0   18081.751261 -3605.898739  0.999968 -1.383040e-07  -0.674919   
    1   18172.456286 -3515.193714  0.999953 -1.987011e-07  -0.978421   
    2   18263.161311 -3424.488689  0.999931 -2.828414e-07  -1.405024   
    3   18353.866336 -3333.783664  0.999901 -3.989928e-07  -1.999039   
    4   18444.571361 -3243.078639  0.999858 -5.579146e-07  -2.818597   
    ..           ...          ...       ...           ...        ...   
    95  26698.728639 -3236.378639 -0.999116 -2.096575e-06 -22.232472   
    96  26789.433664 -3327.083664 -0.999288 -1.708652e-06 -18.259108   
    97  26880.1386

In [26]:
df = strategies_json[datetime.date(2025, 4, 24)]['Long Strangle']['payoff']['payoffs']

In [27]:
df

,S,payoff,delta,gamma,vega,zak_lower_band,zak_upper_band
0,18081.751261,3603.198739,-0.999968,1.383040e-07,0.674919,-1.000152,-0.999784
1,18172.456286,3512.493714,-0.999953,1.987011e-07,0.978421,-1.000154,-0.999752
2,18263.161311,3421.788689,-0.999931,2.828414e-07,1.405024,-1.000153,-0.999710
3,18353.866336,3331.083664,-0.999901,3.989928e-07,1.999039,-1.000146,-0.999655
4,18444.571361,3240.378639,-0.999858,5.579146e-07,2.818597,-1.000131,-0.999584
...,...,...,...,...,...,...,...
95,26698.728639,3233.678639,0.999116,2.096575e-06,22.232472,0.998716,0.999516
96,26789.433664,3324.383664,0.999288,1.708652e-06,18.259108,0.998923,0.999653
97,26880.138689,3415.088689,0.999428,1.388772e-06,14.953933,0.999095,0.999761
98,26970.843714,3505.793714,0.999542,1.125744e-06,12.212835,0.999237,0.999846


In [28]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Ensure that the DataFrame has the required columns:
# 'S', 'payoff', 'delta', 'zak_lower_band', and 'zak_upper_band'

# Create separate columns for payoff above and below zero.
df['payoff_positive'] = df['payoff'].where(df['payoff'] >= 0, np.nan)
df['payoff_negative'] = df['payoff'].where(df['payoff'] < 0, np.nan)

# Create a figure with a secondary y-axis.
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add the positive payoff trace with a green shaded fill.
fig.add_trace(
    go.Scatter(
        x=df['S'],
        y=df['payoff_positive'],
        mode='lines',
        name='Payoff (>= 0)',
        line=dict(color='green'),
        fill='tozeroy',
        fillcolor='rgba(0, 255, 0, 0.3)'
    ),
    secondary_y=False
)

# Add the negative payoff trace with a red line.
fig.add_trace(
    go.Scatter(
        x=df['S'],
        y=df['payoff_negative'],
        mode='lines',
        name='Payoff (< 0)',
        line=dict(color='red')
    ),
    secondary_y=False
)

# Add a horizontal line at y=0.
fig.add_shape(
    type="line",
    x0=df['S'].min(),
    x1=df['S'].max(),
    y0=0,
    y1=0,
    line=dict(color="black", dash="dash"),
    xref="x",
    yref="y"
)

# Plot the additional metrics on the secondary y-axis.
fig.add_trace(
    go.Scatter(
        x=df['S'],
        y=df['delta'],
        mode='lines',
        name='Delta'
    ),
    secondary_y=True
)

fig.add_trace(
    go.Scatter(
        x=df['S'],
        y=df['zak_lower_band'],
        mode='lines',
        name='Zak Lower Band'
    ),
    secondary_y=True
)

fig.add_trace(
    go.Scatter(
        x=df['S'],
        y=df['zak_upper_band'],
        mode='lines',
        name='Zak Upper Band'
    ),
    secondary_y=True
)

# Update layout: set a white background.
fig.update_layout(
    title='DataFrame Plot',
    plot_bgcolor='white',
    paper_bgcolor='white'
)

# Update x-axis: enable grid lines and place the x-axis line (and its tick labels) at y=0.
fig.update_xaxes(
    position=0,          # Places the x-axis at y=0
    showgrid=True,
    gridwidth=1,
    gridcolor='lightgray'
)

# Update y-axes: enable grid lines for both y-axes.
fig.update_yaxes(
    showgrid=True,
    gridwidth=1,
    gridcolor='lightgray',
    secondary_y=False,
    title_text='Payoff'
)
fig.update_yaxes(
    showgrid=True,
    gridwidth=1,
    gridcolor='lightgray',
    secondary_y=True,
    title_text='Other Metrics'
)

fig.show()

### Creating a sample json for testing

In [4]:
with open("options_json_for_ui_testing.pkl", "rb") as file:
    symbols_json = pickle.load(file)

In [5]:
symbols_json

{'DRREDDY': {'underlying': "Dr. Reddy's Laboratories Limited",
  'category': 'equity',
  'realized_volatility': {'Close-to-close': date
   2024-05-15    0.229907
   2024-05-16    0.224621
   2024-05-17    0.225414
   2024-05-21    0.220374
   2024-05-22    0.226026
                   ...   
   2025-03-24    0.222299
   2025-03-25    0.234725
   2025-03-26    0.232751
   2025-03-27    0.231644
   2025-03-28    0.234431
   Name: close, Length: 219, dtype: float64,
   'Yang-Zhang': date
   2024-05-15    0.241643
   2024-05-16    0.252062
   2024-05-17    0.251699
   2024-05-21    0.249321
   2024-05-22    0.250594
                   ...   
   2025-03-24    0.265402
   2025-03-25    0.271249
   2025-03-26    0.271714
   2025-03-27    0.274190
   2025-03-28    0.272785
   Length: 219, dtype: float64,
   'Parkinson': date
   2024-05-14    0.215592
   2024-05-15    0.214654
   2024-05-16    0.221709
   2024-05-17    0.221876
   2024-05-21    0.218246
                   ...   
   2025-03-24   